In [4]:
### НУЖНЫЕ ИМПОРТЫ
# !pip install python-dotenv==1.0.1 mistralai

## 1. USER QUERY

In [5]:
# Пока вопрос в виде строки, потом видимо хотим сделать как .py скрипт
user_query = "Как заселить НИП"

## 2. RETRIEVAL
#### 2.1 BUILDING QUERY EMBEDDINGS

#### 2.2 CHANKING DATA

наверное нужно все данные заранее сделать, а не делать это все в онлайне, просто BERT офигеет столько данных через себя прогонять

#### 2.3 SEARCH

## 3. PROMPT BUILDING

## 4. LLM ANSWER

In [1]:
from TerrariaRAG import TerrariaRAG

terraria_rag = TerrariaRAG()
terraria_rag.set_model("mistral-medium-latest")
terraria_rag.set_temperature(0.1)
question = "Сколько всего кроватей в террария"
answer = terraria_rag.generate_response(question)
print("\n💬 Вопрос:", question)
print("🧠 Ответ:", answer)
print("\n📚 Источники:")

d:\дз\TerrariaRAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing TerrariaRAG components...


d:\дз\TerrariaRAG\src\TerrariaRAG.py:85: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  self.vectorstore = Chroma(persist_directory="./terraria_db", embedding_function=self.embeddings)


TerrariaRAG initialized successfully.

💬 Вопрос: Сколько всего кроватей в террария
🧠 Ответ: У меня нет точных данных о **полном количестве уникальных кроватей** в *Terraria* в текущей базе знаний.

Однако могу привести **основные категории и примеры** кроватей, которые существуют в игре (на основе стандартного контента без модов):

### 🛏️ **Типы кроватей в Terraria**
1. **Стандартные кровати** (крафтятся из дерева и ткани):
   - Обычная кровать (Wood + Silk)
   - Вариации по биомам (например, *Джунглевая кровать*, *Кровати из грибного дерева* и т.д.).

2. **Кровати из разных материалов** (например, из мебели биомов):
   - *Кровать из кактуса* (Cactus)
   - *Кровать из тыквы* (Pumpkin)
   - *Кровать из льда* (Ice)
   - *Кровать из паутины* (Cobweb)
   - *Кровати из мебели Подземелья, Адского камня, Хлорофита* и др.

3. **Тематические кровати** (связанные с событиями или боссами):
   - *Кровать из тыквы* (Halloween)
   - *Кровать из ледяных блоков* (Frost Moon)
   - *Кровати из мебели Лу

In [2]:
terraria_rag.get_models()

['mistral-medium-2505',
 'mistral-large-latest',
 'mistral-medium-2508',
 'mistral-medium-latest',
 'mistral-medium',
 'ministral-3b-2410',
 'ministral-3b-latest',
 'ministral-8b-2410',
 'ministral-8b-latest',
 'open-mistral-7b',
 'mistral-tiny',
 'mistral-tiny-2312',
 'open-mistral-nemo',
 'open-mistral-nemo-2407',
 'mistral-tiny-2407',
 'mistral-tiny-latest',
 'mistral-large-2411',
 'pixtral-large-2411',
 'pixtral-large-latest',
 'mistral-large-pixtral-2411',
 'codestral-2508',
 'codestral-latest',
 'devstral-small-2507',
 'devstral-small-latest',
 'devstral-medium-2507',
 'devstral-medium-latest',
 'mistral-vibe-cli-latest',
 'pixtral-12b-2409',
 'pixtral-12b',
 'pixtral-12b-latest',
 'mistral-small-2506',
 'mistral-small-latest',
 'magistral-medium-2509',
 'magistral-medium-latest',
 'magistral-small-2509',
 'magistral-small-latest',
 'voxtral-mini-2507',
 'voxtral-mini-latest',
 'voxtral-small-2507',
 'voxtral-small-latest',
 'devstral-small-2505',
 'magistral-small-2506',
 'magis